In [1]:
import mlflow
import dotenv
import os
import pandas as pd

In [2]:
dotenv.load_dotenv()
mlflow.set_tracking_uri(os.getenv('MLFLOW_TRACKING_URI'))

## 1. List all latest model versions, and Aliasis

In [12]:
client = mlflow.MlflowClient()
registered_models = client.search_registered_models()

for rm in registered_models:
    print(f"Model Name: {rm.name}")
    
    # Get the latest versions
    if rm.latest_versions:
        latest = rm.latest_versions[0]
        print(f"  Latest Version: {latest.version}, Alias(es): {rm.aliases}")

Model Name: FinanceModel
  Latest Version: 2, Alias(es): {'challenger': '2', 'champion': '1'}
Model Name: TestModelname
  Latest Version: 8, Alias(es): {'champion': '8'}


## 2.1 Download the Model artifacts to local directory

In [14]:
DST_DIR = "imported_artifacts"

local_path = mlflow.artifacts.download_artifacts(
    artifact_uri=f"models:/FinanceModel@challenger", # Default to 'latest'
    dst_path=DST_DIR
)
print(f"Model downloaded to: {local_path}")

Model downloaded to: /Users/rasmus/Ohjelmointi/vscode/mlflow-model-gcp/imported_artifacts/


## 2.2 Load the Model object from the local artifacts

In [15]:
model = mlflow.pyfunc.load_model(local_path)

2025/12/04 21:19:16 WARNING mlflow.utils.requirements_utils: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - pip (current: uninstalled, required: pip>=24.0)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.


## 3. Load Model object directly from server

In [16]:
model = mlflow.pyfunc.load_model("models:/FinanceModel@challenger")

2025/12/04 21:21:04 WARNING mlflow.utils.requirements_utils: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - pip (current: uninstalled, required: pip>=24.0)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.


## 4. Model attributes

In [27]:
model

mlflow.pyfunc.loaded_model:
  artifact_path: gs://mlflow-tracking-gcs/4/models/m-6fcd95b8621b491e8f76e308621380b5/artifacts
  flavor: mlflow.pyfunc.loaders.code_model
  run_id: 2113e9d21c234c6d9836718141d524e6

In [21]:
model.metadata.signature

inputs: 
  ['receiver': string (required), 'amount': double (required)]
outputs: 
  [string (required)]
params: 
  None

In [ ]:
df = model.input_example # The logged input example
display(df)

## 5. Predict

In [28]:
model.predict(df)

array(['SALARY', 'FOOD', 'LIVING', 'ENTERTAINMENT', 'ENTERTAINMENT'],
      dtype=object)

In [ ]:
df2 = df.copy()
df2['NewColumn'] = 'Testing123' # New features are ignored by the model

model.predict(df2)

2025/12/04 21:27:10 WARNING mlflow.models.utils: Found extra inputs in the model input that are not defined in the model signature: `['NewColumn']`. These inputs will be ignored.


array(['SALARY', 'FOOD', 'LIVING', 'ENTERTAINMENT', 'ENTERTAINMENT'],
      dtype=object)

In [ ]:
df3 = df.copy()
df3 = df3.drop(columns=['Amount']) # The model automatically validates input for signature compliance

model.predict(df3)

KeyError: "['Amount'] not found in axis"